In [8]:
from src.crawler import WebCrawler
from src.parser import VerbParser
import pandas as pd

crawler = WebCrawler()
parser = VerbParser()

url = "https://www.spanishdict.com/conjugate/gustar"

html = crawler.crawl_page(url)
parser.set_html_soup(html)

In [9]:
doc = parser.html_soup
tables = pd.read_html(html)
tables

[          0         1
 0  Present:  gustando
 1     Past:   gustado,
                   0         1           2           3            4           5
 0               NaN   Present   Preterite   Imperfect  Conditional      Future
 1                yo     gusto       gusté     gustaba     gustaría     gustaré
 2                tú    gustas    gustaste    gustabas    gustarías    gustarás
 3       él/ella/Ud.     gusta       gustó     gustaba     gustaría     gustará
 4          nosotros  gustamos    gustamos  gustábamos  gustaríamos  gustaremos
 5          vosotros   gustáis  gustasteis   gustabais   gustaríais   gustaréis
 6  ellos/ellas/Uds.    gustan    gustaron    gustaban    gustarían    gustarán,
                   0         1                       2           3
 0               NaN   Present               Imperfect      Future
 1                yo     guste        gustara, gustase     gustare
 2                tú    gustes      gustaras, gustases    gustares
 3       él/ella/Ud. 